Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from assets/wikipedia_data.html and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in assets/nfl.csv), MLB (baseball, in assets/mlb.csv), NBA (basketball, in assets/nba.csv or NHL (hockey, in assets/nhl.csv). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: what is the win/loss ratio's correlation with the population of the city it is in? Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with pearsonr, so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%*4=80%) of the grade for this assignment. You should only use data from year 2018 for your analysis -- this is important!

Notes
Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!
Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NHL using 2018 data.

In [4]:
import pandas as pd
import numpy as np
import codecs
import scipy.stats as stats
import re

with open("/Users/williamwoodfine/Downloads/Coursera Assignments/Assignment 4/wikipedia_data.html", "r") as file:
    wiki=file.read()
wiki=re.split("wikitable sortable jquery-tablesorter",wiki)[1]
def splittorow(group):
    wikisec=list(re.split("<tr>",wiki))[group]
    df_new=pd.Series(re.split("<td>|<td",wikisec))
    return df_new
cols=[x.group(2) for x in re.finditer("(ascending\">|area\">)([\w\d\s]*)", list(re.split("<tr>",wiki))[1])][1:]
cols=[re.sub("\\n","", item).strip() for item in cols]
df=pd.DataFrame()
for i in range(52):
    row=splittorow(i)
    df=df.append(row, ignore_index=True)
df=df.drop(index=[0,1], columns=0)
df.columns=cols
df["Metropolitan area"]=df["Metropolitan area"].replace("^[^:]*:[^>]*>", "", regex=True).replace("</a>\\n</td>\\n$", "", regex=True).replace("</a></span>\\n</td>\n", "", regex=True)
df["Country"]=df["Country"].replace("\\n</td>\\n", "", regex=True)
df["Pop"]=df["Pop"].replace("\\n</td>\\n","", regex=True)
df["Population"]=df["Population"].replace("\\n</td>\\n", "", regex=True)
df["B4"]=df["B4"].replace("\n</td>\n", "", regex=True)
df["B6"]=df["B6"].replace("\n</td>\n", "", regex=True)
def isolate_team(txt):
    return [x[1] for x in re.findall("(title=\")([^\"]*)", txt)]
df["NFL"]=df["NFL"].apply(isolate_team)
df["NBA"]=df["NBA"].apply(isolate_team)
df["NHL"]=df["NHL"].apply(isolate_team)
df["MLS"]=df["MLS"].apply(isolate_team)
df["MLB"]=df["MLB"].apply(isolate_team)
df["CFL"]=df["CFL"].apply(isolate_team)
df["Population"]=df["Population"].apply(lambda x: int(x.replace(",","")))
df=df[["Metropolitan area","Population", "NHL"]]
df = df.rename(columns={"NHL":"team"}).explode("team").dropna()
df_cities=df

#Prepare STATS for NHL   
df=pd.read_csv("/Users/williamwoodfine/Downloads/Coursera Assignments/Assignment 4/nhl.csv")
df=df[df["year"]==2018]
divisions=["Atlantic Division", "Metropolitan Division","Central Division","Pacific Division"]
mask=df["team"].apply(lambda x: bool(x in divisions))
def teamnames(item):
    if item ==1:
        return divisions[0]
    if item ==2:
        return divisions[1]
    if item ==3:
        return divisions[2]
    if item ==4:
        return divisions[3]
    
df["division"]=np.cumsum(mask).apply(teamnames)
df["mask"]=mask
df=df.set_index("mask").drop(True, axis=0).reset_index().drop("mask", axis=1)
df["team"]=df["team"].replace("\*", "", regex=True)
df["W"]=df["W"].apply(lambda x:int(x))
df["L"]=df["L"].apply(lambda x:int(x))

df_stats=df






def nhl_correlation(): 
    df=pd.merge(df_cities, df_stats, how="inner", left_on="team",right_on="team")

    def find_ratio(group):    
        total_wins= np.sum(group["W"])
        total_losses=np.sum(group["L"])    
        return total_wins/(total_wins+total_losses)

    df=df.groupby(by=["Metropolitan area", "Population"]).apply(find_ratio)
    df=pd.DataFrame(df).reset_index().rename(columns={0:"Wins to Losses"}).set_index("Metropolitan area")
    
    
    population_by_region= [x for x in df["Population"]]
    win_loss_by_region = [x for x in df["Wins to Losses"]]
    
    

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
nhl_correlation()

0.012308996455744264

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NBA using 2018 data.


In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


with open("/Users/williamwoodfine/Downloads/Coursera Assignments/Assignment 4/wikipedia_data.html", "r") as file:
    wiki=file.read()
wiki=re.split("wikitable sortable jquery-tablesorter",wiki)[1]
def splittorow(group):
    wikisec=list(re.split("<tr>",wiki))[group]
    df_new=pd.Series(re.split("<td>|<td",wikisec))
    return df_new
cols=[x.group(2) for x in re.finditer("(ascending\">|area\">)([\w\d\s]*)", list(re.split("<tr>",wiki))[1])][1:]
cols=[re.sub("\\n","", item).strip() for item in cols]
df=pd.DataFrame()
for i in range(52):
    row=splittorow(i)
    df=df.append(row, ignore_index=True)
df=df.drop(index=[0,1], columns=0)
df.columns=cols
df["Metropolitan area"]=df["Metropolitan area"].replace("^[^:]*:[^>]*>", "", regex=True).replace("</a>\\n</td>\\n$", "", regex=True).replace("</a></span>\\n</td>\n", "", regex=True)
df["Country"]=df["Country"].replace("\\n</td>\\n", "", regex=True)
df["Pop"]=df["Pop"].replace("\\n</td>\\n","", regex=True)
df["Population"]=df["Population"].replace("\\n</td>\\n", "", regex=True)
df["B4"]=df["B4"].replace("\n</td>\n", "", regex=True)
df["B6"]=df["B6"].replace("\n</td>\n", "", regex=True)
def isolate_team(txt):
    return [x[1] for x in re.findall("(title=\")([^\"]*)", txt)]
df["NFL"]=df["NFL"].apply(isolate_team)
df["NBA"]=df["NBA"].apply(isolate_team)
df["NHL"]=df["NHL"].apply(isolate_team)
df["MLS"]=df["MLS"].apply(isolate_team)
df["MLB"]=df["MLB"].apply(isolate_team)
df["CFL"]=df["CFL"].apply(isolate_team)
df["Population"]=df["Population"].apply(lambda x: int(x.replace(",","")))
df=df[["Metropolitan area","Population", "NBA"]]
df = df.rename(columns={"NBA":"team"}).explode("team").dropna()
df_cities=df

#Prepare STATS for NHL   
df=pd.read_csv("/Users/williamwoodfine/Downloads/Coursera Assignments/Assignment 4/nba.csv")
df=df[df["year"]==2018]
df["team"]=df["team"].replace("\(\d{0,2}\)", "", regex=True).replace("\*", "", regex=True).str.strip()
df["W"]=df["W"].apply(lambda x:int(x))
df["L"]=df["L"].apply(lambda x:int(x))
df_stats=df







def nba_correlation():
    df=pd.merge(df_cities, df_stats, how="inner", left_on="team",right_on="team")

    def find_ratio(group):    
        total_wins= np.sum(group["W"])
        total_losses=np.sum(group["L"])    
        return total_wins/(total_wins+total_losses)

    df=df.groupby(by=["Metropolitan area", "Population"]).apply(find_ratio)
    df=pd.DataFrame(df).reset_index().rename(columns={0:"Wins to Losses"}).set_index("Metropolitan area")

    population_by_region= [x for x in df["Population"]]
    win_loss_by_region = [x for x in df["Wins to Losses"]]
  
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"
    

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()

nba_correlation()   

-0.17657160252844611

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the MLB using 2018 data.

import pandas as pd

In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

with open("/Users/williamwoodfine/Downloads/Coursera Assignments/Assignment 4/wikipedia_data.html", "r") as file:
    wiki=file.read()
wiki=re.split("wikitable sortable jquery-tablesorter",wiki)[1]
def splittorow(group):
    wikisec=list(re.split("<tr>",wiki))[group]
    df_new=pd.Series(re.split("<td>|<td",wikisec))
    return df_new
cols=[x.group(2) for x in re.finditer("(ascending\">|area\">)([\w\d\s]*)", list(re.split("<tr>",wiki))[1])][1:]
cols=[re.sub("\\n","", item).strip() for item in cols]
df=pd.DataFrame()
for i in range(52):
    row=splittorow(i)
    df=df.append(row, ignore_index=True)
df=df.drop(index=[0,1], columns=0)
df.columns=cols
df["Metropolitan area"]=df["Metropolitan area"].replace("^[^:]*:[^>]*>", "", regex=True).replace("</a>\\n</td>\\n$", "", regex=True).replace("</a></span>\\n</td>\n", "", regex=True)
df["Country"]=df["Country"].replace("\\n</td>\\n", "", regex=True)
df["Pop"]=df["Pop"].replace("\\n</td>\\n","", regex=True)
df["Population"]=df["Population"].replace("\\n</td>\\n", "", regex=True)
df["B4"]=df["B4"].replace("\n</td>\n", "", regex=True)
df["B6"]=df["B6"].replace("\n</td>\n", "", regex=True)
def isolate_team(txt):
    return [x[1] for x in re.findall("(title=\")([^\"]*)", txt)]
df["NFL"]=df["NFL"].apply(isolate_team)
df["NBA"]=df["NBA"].apply(isolate_team)
df["NHL"]=df["NHL"].apply(isolate_team)
df["MLS"]=df["MLS"].apply(isolate_team)
df["MLB"]=df["MLB"].apply(isolate_team)
df["CFL"]=df["CFL"].apply(isolate_team)
df["Population"]=df["Population"].apply(lambda x: int(x.replace(",","")))
df=df[["Metropolitan area","Population", "MLB"]]
df = df.rename(columns={"MLB":"team"}).explode("team").dropna()
df["team"]=df["team"].replace("\(baseball\)" , "", regex=True).str.strip()
df_cities=df


df=pd.read_csv("/Users/williamwoodfine/Downloads/Coursera Assignments/Assignment 4/mlb.csv")
df=df[df["year"]==2018]
df["team"]=df["team"].replace("\(\d{0,2}\)", "", regex=True).replace("\*", "", regex=True).str.strip()
df["W"]=df["W"].apply(lambda x:int(x))
df["L"]=df["L"].apply(lambda x:int(x))
df_stats=df




def mlb_correlation(): 
    df=pd.merge(df_cities, df_stats, how="outer", left_on="team",right_on="team", indicator=True)

    def find_ratio(group):    
        total_wins= np.sum(group["W"])
        total_losses=np.sum(group["L"])    
        return total_wins/(total_wins+total_losses)

    df=df.groupby(by=["Metropolitan area", "Population"]).apply(find_ratio)
    df=pd.DataFrame(df).reset_index().rename(columns={0:"Wins to Losses"}).set_index("Metropolitan area")

    population_by_region= [x for x in df["Population"]]
    win_loss_by_region = [x for x in df["Wins to Losses"]]

    df
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
    
mlb_correlation()

0.1505230448710485

For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the NFL using 2018 data.



In [9]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

with open("/Users/williamwoodfine/Downloads/Coursera Assignments/Assignment 4/wikipedia_data.html", "r") as file:
    wiki=file.read()
wiki=re.split("wikitable sortable jquery-tablesorter",wiki)[1]
def splittorow(group):
    wikisec=list(re.split("<tr>",wiki))[group]
    df_new=pd.Series(re.split("<td>|<td",wikisec))
    return df_new
cols=[x.group(2) for x in re.finditer("(ascending\">|area\">)([\w\d\s]*)", list(re.split("<tr>",wiki))[1])][1:]
cols=[re.sub("\\n","", item).strip() for item in cols]
df=pd.DataFrame()
for i in range(52):
    row=splittorow(i)
    df=df.append(row, ignore_index=True)
df=df.drop(index=[0,1], columns=0)
df.columns=cols
df["Metropolitan area"]=df["Metropolitan area"].replace("^[^:]*:[^>]*>", "", regex=True).replace("</a>\\n</td>\\n$", "", regex=True).replace("</a></span>\\n</td>\n", "", regex=True)
df["Country"]=df["Country"].replace("\\n</td>\\n", "", regex=True)
df["Pop"]=df["Pop"].replace("\\n</td>\\n","", regex=True)
df["Population"]=df["Population"].replace("\\n</td>\\n", "", regex=True)
df["B4"]=df["B4"].replace("\n</td>\n", "", regex=True)
df["B6"]=df["B6"].replace("\n</td>\n", "", regex=True)
def isolate_team(txt):
    return [x[1] for x in re.findall("(title=\")([^\"]*)", txt)]
df["NFL"]=df["NFL"].apply(isolate_team)
df["NBA"]=df["NBA"].apply(isolate_team)
df["NHL"]=df["NHL"].apply(isolate_team)
df["MLS"]=df["MLS"].apply(isolate_team)
df["MLB"]=df["MLB"].apply(isolate_team)
df["CFL"]=df["CFL"].apply(isolate_team)
df["Population"]=df["Population"].apply(lambda x: int(x.replace(",","")))
df=df[["Metropolitan area","Population", "NFL"]]
df = df.rename(columns={"NFL":"team"}).explode("team").dropna()
df_cities=df


df=pd.read_csv("/Users/williamwoodfine/Downloads/Coursera Assignments/Assignment 4/nfl.csv")
df=df[df["year"]==2018]
divisions=["AFC East","AFC North","AFC South","AFC West","NFC East","NFC North","NFC South","NFC West"]
mask=df["team"].apply(lambda x: bool(x in divisions))
def teamnames(item):
    if item ==1:
        return divisions[0]
    if item ==2:
        return divisions[1]
    if item ==3:
        return divisions[2]
    if item ==4:
        return divisions[3]
df["division"]=np.cumsum(mask).apply(teamnames)
df["mask"]=mask
df=df.set_index("mask").drop(True, axis=0).reset_index().drop("mask", axis=1)
df["team"]=df["team"].replace("\+|\*", "", regex=True)
df["W"]=df["W"].apply(lambda x:int(x))
df["L"]=df["L"].apply(lambda x:int(x))
df_stats=df



def nfl_correlation(): 
    df=pd.merge(df_cities, df_stats, how="inner", left_on="team",right_on="team")


    def find_ratio(group):    
        total_wins= np.sum(group["W"])
        total_losses=np.sum(group["L"])    
        return total_wins/(total_wins+total_losses)

    df=df.groupby(by=["Metropolitan area", "Population"]).apply(find_ratio)
    df=pd.DataFrame(df).reset_index().rename(columns={0:"Wins to Losses"}).set_index("Metropolitan area")

    population_by_region= [x for x in df["Population"]]
    win_loss_by_region = [x for x in df["Wins to Losses"]]
    
       
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError() 
    
nfl_correlation()

0.004922112149349456